In [ ]:
import pandas as pd
import glob
import os
import re
import random

# Bring all CSV file paths in the path
path = '/content/drive/MyDrive/Kwargs/적합성/*.csv'
all_files = glob.glob(path)

# Initialization of data frame list to save the read CSV files
dfs = []

# Read each file and add it to the data frame list
for filename in all_files:
# Extracting company name in the file path (e.g. "Company_related_KB Finance .csv" -> "KB Finance")
    company_name = os.path.basename(filename).split('_')[-1].split('.')[0]

# Create a data frame by reading the CSV file
    df = pd.read_csv(filename)

# Add a company name extracted to the new column 'company name'
    df['기업명'] = company_name

# Title and Content column combine the contents of the full_text column overwritten
    df['full_text'] = df['title'] + '  ' + df['content']

# Add to data frame list
    dfs.append(df)




                                           full_text           기업명
0  스페셜 리포트] 한국에는 ‘CEO 시장’이 없다  두 부문은 2014년까지 통합 운...  삼성물산
1  고의 공시누락?…근거없다 법정서 맞붙은 삼바-증선위  이날 심문은 삼성바이오로직스가...  삼성물산
2  사라지는 로또아파트·깨지는 분양공식…'대마불사'는 통할까  지난 달에는 서울에서는 ...  삼성물산
3  한진칼에만 최소한 경영참여…반쪽짜리 절충안 나온 이유는  1일 국민연금의 최고의사결...  삼성물산
4  마켓뷰] 뒷심 발휘한 코스피…1월에만 3.4조원 산 외국인  반면 코스닥지수는 4거...  삼성물산


In [ ]:
# All data frame is merged into one (ignoring indexes and joining)
original_df = pd.concat(dfs, ignore_index=True)

# Output (only 5 rows of rows)
print(original_df[['full_text', '기업명']].head())

                                           full_text           기업명
0  스페셜 리포트] 한국에는 ‘CEO 시장’이 없다  두 부문은 2014년까지 통합 운...  삼성물산
1  고의 공시누락?…근거없다 법정서 맞붙은 삼바-증선위  이날 심문은 삼성바이오로직스가...  삼성물산
2  사라지는 로또아파트·깨지는 분양공식…'대마불사'는 통할까  지난 달에는 서울에서는 ...  삼성물산
3  한진칼에만 최소한 경영참여…반쪽짜리 절충안 나온 이유는  1일 국민연금의 최고의사결...  삼성물산
4  마켓뷰] 뒷심 발휘한 코스피…1월에만 3.4조원 산 외국인  반면 코스닥지수는 4거...  삼성물산


In [ ]:
original_df['기업명'].value_counts()

기업명
SK하이닉스            8131
삼성바이오로직스    8127
삼성SDI              7807
삼성물산           7787
현대차                6814
카카오                 6299
신한지주             5666
NAVER                  5589
KB금융               5416
기아                   5333
삼성생명           5071
현대모비스            5068
삼성전자            4385
포스코 홀딩스        4278
Name: count, dtype: int64

In [ ]:
# Creation of example data
#data = {
# 'Company name': [Samsung Electronics', 'Hyundai Motor', 'LG Electronics',' SK Innovation ',' Kakao ',' Samsung Electronics', 'Kakao', 'LG Electronics',' SK Innovation ',' Hyundai Motor ']
# 'Title': [Title 1, 'Title 2', 'Title 3', 'Title 4', 'Title 5', 'Title 6', 'Title 7', 'Title 8', 'Title 9', 'Title 10']
# 'Content': [
# 'Samsung Electronics and Hyundai Motor are developing innovative technology, and LG Electronics has also entered this competition.'
# 'SK Innovation is focusing on developing eco -friendly energy, and Kakao is leading digital innovation.'
# 'Samsung has launched the latest smartphone model, and LG Electronics unveiled its new home appliance lineup. Hyundai is focusing on autonomous driving technology. '
# 'Kakao has recently invested in the fintech sector, and SK Innovation is speeding up battery technology innovation.'
# 'Hyundai Motor is leading the electric vehicle market, and Samsung Electronics and LG Electronics have begun to cooperate with battery technology.'
# 'LG Electronics launched eco -friendly home appliances, and SK Innovation introduced recycled battery technology.'
# 'Samsung Electronics is investing huge investments in artificial intelligence research, and Hyundai is developing smart mobility solutions.'
# 'Kakao is showing interest in blockchain technology recently, and LG Electronics is developing smart home technology.'
# '' SK Innovation and Hyundai Motor Company are collaborating in the study of electric vehicle battery technology. '
# 'Samsung Electronics has an advantage in the semiconductor market, and LG Electronics and Kakao are developing new business models.'
#]
#}

In [ ]:


combined_df = original_df.copy()

In [ ]:
import pandas as pd
import random
import re
import unicodedata
from tqdm import tqdm

# Definition of the prefix of the Co., Ltd.
prefix_suffix_patterns = [
    r'\b주식회사\b', r'\b\(주\)\b', r'㈜', r'\(.*?\)'  # 괄호와 괄호 안의 내용 제거
]


# Company name unification mapping
company_name_mapping = {
    'SK하이닉스': 'SK하이닉스',
    '에스케이하이닉스': 'SK하이닉스',
    '삼성바이오로직스': '삼성바이오로직스',
    '삼바': '삼성바이오로직스',
    '삼성바이오': '삼성바이오로직스',
    '삼성SDI': '삼성SDI',
    '삼성에스디아이':'삼성SDI',
    '삼성물산': '삼성물산',
    '현대차': '현대차',
    '현대자동차': '현대차',
    '카카오': '카카오',
    '신한지주': '신한지주',
    'NAVER': '네이버',
    '네이버': '네이버',
    'KB금융': 'KB금융',
    '국민은행':'KB금융',
    '기아': '기아',
    '기아차':'기아',
    '기아자동차':'기아',
    '삼성생명': '삼성생명',
    '현대모비스': '현대모비스',
    '삼성전자': '삼성전자',
    'POSCO':'POSCO홀딩스',
    '포스코홀딩스': 'POSCO홀딩스',
    'POSCO홀딩스': 'POSCO홀딩스'
}

def normalize_text(text):
    """텍스트를 유니코드 NFC 형식으로 정규화"""
    return unicodedata.normalize('NFC', text)

def clean_company_name(name):
    """주식회사 접두사/접미사 및 괄호와 그 안의 내용 제거하여 기업명 표준화"""
    cleaned_name = normalize_text(name)  # 유니코드 정규화
    for pattern in prefix_suffix_patterns:
        cleaned_name = re.sub(pattern, '', cleaned_name)
    cleaned_name = cleaned_name.strip()  # 앞뒤 공백 제거
# If you have a mapped name, it is returned to a unified name
    return company_name_mapping.get(cleaned_name, cleaned_name)

def load_and_clean_company_names():
    """기존 데이터와 CSV 파일에서 모든 기업명 로드 및 중복 제거 후 정제"""
# Extracting unique company name from existing combined_df
    combined_df_names = combined_df['기업명'].unique().tolist()

# 'CompanyNames.csv' file load
    company_names_df = pd.read_csv('companynames.csv')

# Extract all the company names of 'CompanyNames.csv' as a list
    additional_company_names = company_names_df.iloc[:, 0].tolist()  # 첫 번째 컬럼 추출

# Merge of the existing unique company name list and additional company name list
    all_company_names = combined_df_names + additional_company_names

#Duplicate removal and refined name return
    cleaned_names = list(set(clean_company_name(name) for name in all_company_names))

# Remove the name in the inclusion
    final_names = remove_contained_names(cleaned_names)

    return final_names

def remove_contained_names(names):
    """부분적으로 포함되는 이름을 제거하여 최종 기업명 리스트 생성"""
    final_names = set(names)
    for name in names:
        for other in names:
            if name != other and name in other:
                final_names.discard(name)
                break
    return list(final_names)

def anonymize_company_names(df, all_companies):
# Create anonymization name using alphabet from A to Z
    alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

# Dictionary to track the anonymized name already used for each company
    company_used_anonymous = {clean_company_name(company): set() for company in all_companies}

# Trace an anonymized name for each article so that it is not duplicated
    global_used_anonymous = set()

    for i, row in tqdm(df.iterrows(), total=len(df), desc="익명화 진행 중"):
        content = normalize_text(row['full_text'])  # 기사 내용 정규화

# Matching even if there are characters, blanks, and special characters before and after the company name
        companies_in_article = list(set(re.findall(r'(?<!\w)(' + '|'.join([re.escape(comp) for comp in all_companies]) + r')(?!\w)', content)))

# Treatment to include the company name in the company name column
        original_company_name = clean_company_name(str(row['기업명']))
        if original_company_name not in companies_in_article:
            companies_in_article.append(original_company_name)

#Ner knight's unique anonymization mapping dictionary
        company_to_anonymous = {}
        used_letters = set(global_used_anonymous)  # 전역 사용 익명화 이름 사용 추적

        for company in companies_in_article:
            normalized_company = clean_company_name(company)

# ** If the company name does not exist in advance, initialization with an empty set **
            if normalized_company not in company_used_anonymous:
                company_used_anonymous[normalized_company] = set()

            if normalized_company not in company_to_anonymous:
# Trace the anonymization name already used for the company
                already_used = company_used_anonymous[normalized_company]

#Select anonymization name possible (new name is first)
                available_letters = list(set(alphabet) - used_letters - already_used)

                if not available_letters:  # 모든 알파벳이 사용된 경우, 중복 사용 허용
                    available_letters = list(set(alphabet) - already_used)

                random_letter = random.choice(available_letters)
                anonymous_name = f'{random_letter}사'

                company_to_anonymous[normalized_company] = anonymous_name
                used_letters.add(random_letter)
                company_used_anonymous[normalized_company].add(anonymous_name)  # 기업별 사용된 이름 기록
                global_used_anonymous.add(anonymous_name)  # 전역 사용된 이름 기록

        if i % 1000 == 0:
            print(f"Row {i} 기업명 -> 익명화 이름 매핑: {company_to_anonymous}")  # 디버깅 출력

#Interacted the company's name in an anonymized name
        for company in companies_in_article:
            normalized_company = clean_company_name(company)
            content = re.sub(r'(?<!\w)' + re.escape(company) + r'(?!\w)', company_to_anonymous[normalized_company], content)

# Update the article and the company name with anonymized name
        df.at[i, 'full_text'] = content

# Change the value of the corporate name column to an anonymous name
        df.at[i, '기업명'] = company_to_anonymous.get(original_company_name, original_company_name)

        if i % 1000 == 0:
            print(f"Row {i} after anonymization - 기업명: {df.at[i, '기업명']}, full_text: {df.at[i, 'full_text']}")  # 디버깅 출력

    return df

# Road and refine all corporate names
all_companies = load_and_clean_company_names()

#Annation of anonymization
df = anonymize_company_names(combined_df, all_companies)

print("\n익명화된 데이터프레임:")
print(df.head())  # 결과를 확인하기 위해 첫 5개 행만 출력


익명화 진행 중:   0%|          | 13/85771 [00:00<27:54, 51.21it/s] 

Row 0 기업명 -> 익명화 이름 매핑: {'현대차': 'I사', '동원산업': 'K사', '현대백화점': 'H사', '대우조선해양': 'S사', '삼성물산': 'C사', '한화케미칼': 'N사', 'LG화학': 'O사', '삼성전자': 'L사', 'KCC': 'W사', 'CJ제일제당': 'D사', 'CJ대한통운': 'Q사', '신세계': 'J사', '태광산업': 'M사'}
Row 0 after anonymization - 기업명: C사, full_text: 스페셜 리포트] 한국에는 ‘CEO 시장’이 없다  두 부문은 2014년까지 통합 운영돼오다가 2015년 각 부문의 전문성 제고를 위해 분리됐다가 3년 만에 재통합됐다.  한화그룹이 외부 인사에게 화약과 방산부문 수장이란 중책을 맡긴 것을 두고 이례적이라는 평이 지배적이다.  옥 사장은 삼성전자에서 LCD사업부 지원팀장.  DS. 디바이스솔루션. 부문 경영지원실장을 거쳐 부사장까지 오른 인물이다.  2016년 한화그룹에 합류해 N사 폴리실리콘사업부 사장.  한화건설 관리담당 사장을 거쳤고 2017년엔 비. 非. 한화 출신 인사로는 처음으로 화약부문 대표를 맡아 주목을 받았다.  김승연 한화그룹 회장이 그룹의 핵심 사업을 삼성 출신에게 맡긴 것을 두고 화약. 방산부문을 성장시키겠다는 의지가 반영된 것이라는 해석이 나온다.  지난해 말 현대차그룹은 부회장 및 사장단 인사 교체를 대대적으로 단행했다.  이 인사 중 가장 돋보였던 인물은 알버트 비어만 현대. 기아차 차량성능담당 사장이다.  그는 신임 연구개발본부장을 맡게 됐다.  외국인이 연구개발본부장에 오른 첫 사례였다.  비어만 본부장은 독일 BMW 출신으로 2015년 현대차그룹에 합류했다.  이후 I사 시험. 고성능차 개발담당. 부사장.  등을 거쳤다.  이에 앞서 LG그룹 역시 지난해 11월 O사 신임 대표이사 부회장에 3M 출신 신학철 수석부회장을 영입해 업계의 주목을 받았다.  외부 영입 CEO 25. 에 불과 재계에서 이른바 . 순혈주의. 가 무너지

익명화 진행 중:   1%|          | 1018/85771 [00:06<09:58, 141.61it/s]

Row 1000 기업명 -> 익명화 이름 매핑: {'삼성물산': 'W사', '삼성전자': 'I사'}
Row 1000 after anonymization - 기업명: W사, full_text: W사 저평가는 맞는데…회복은 험난  그러나 회복까지는 험난할 수 있다는 전망이다.  3일 오전 10시50분 삼성물산은 전날보다 500원. 0.  58. .  오른 8만7100원에 거래됐다.  지난달 7일 8만4100원에 거래되며 52주 신저가를 경신한 뒤 9만원대를 곧 회복하는 듯 했으나 이후 국정농단 사건에 연루된 이 부회장이 재판을 다시 받게 되면서 하락해 저점 근처를 맴돌고 있다.  이 부회장은 앞서 박근혜 전 대통령과 최순실씨 일가에 Δ미르. K스포츠재단 204억원 Δ한국동계스포츠영재센터 16억2800만원 Δ정유라 승마지원 77억9735만원. 약속 금액 213억원.  등 433억2800만원의 뇌물을 주거나 약속한 혐의로 구속 기소됐다.  이 부회장은 2017년 8월 1심에서 징역 5년 실형을 선고받고 법정 구속됐으나 지난해 2월 2심에서 징역 2년 6개월에 집행유예 4년을 받고 풀려났다.  대법원은 그러나 삼성이 최순실씨의 딸 정유라씨에게 지원한 말 3마리의 소유권이 최씨에게 넘어간 만큼 뇌물로 봐야 한다고 판단했다.  여기에 2심과 달리 한국동계스포츠영재센터 지원금 16억여원도 뇌물로 판단했다.  특히 대법원이 이 부회장의 승계 작업을 위한 부정한 청탁이 인정된다고 밝혀 지배구조의 핵심인 W사 주가에 타격을 줬다.  W사 주가는 2016년 국정농단에 대한 수사가 시작되며 외부 변수에 영향을 많이 받았다.  조윤호 DB금융투자 연구원은 2017. 2018년 실적 턴어라운드에도 불구하고 지배구조 최상단 기업으로서 실적보다는 그룹 관련 이슈가 주가를 지배했다며 대법원 최종 판결로 불확실성이 감소할 것을 기대했지만 결과는 그렇지 못했다고 지적했다.  이같은 논란을 고려해도 증권업계에서는 삼성물산의 현 주가 수준은 지나친 저평가라고 입을 모은다.  백광제 교보증권 연구원은

익명화 진행 중:   2%|▏         | 2024/85771 [00:14<11:33, 120.77it/s]

...

Row 84000 기업명 -> 익명화 이름 매핑: {'대상': 'P사', '포스코 홀딩스': 'E사'}
Row 84000 after anonymization - 기업명: E사, full_text: 성과 없다는 지적에…국내 기업 부담 최소화 합의  양국 주무 장관이 만나 공동선언문을 발표했지만.  원론적인 협의를 하겠다는 정도에서 별다른 진전은 없습니다.  신성우 기자.  정부가 해결책을 들고 오지 못했다는 지적에 대해 어떻게 설명하고 있나요. 기자대통령실은 정상회담은 구체적인 내용을 합의하는 자리가 아니라며 공통적인 인식을 확인하는 것이 중요하다고 밝혔습니다.  이어 한국 기업의 부담과 불확실성을 줄여준다는 방향에 대해 명확하게 합의를 했다며.  바이든 대통령의 명확한 지침이 확인됐다고 설명했는데요.  명확한 해결책이 발표된 것은 아니지만.  방향성을 공유했다는 것입니다.  다만 기술적이고 세부적인 국가 간의 협의가 필요하다고 부연 설명했는데요.  사실상 과제가 남았음을 시사한 것입니다.  앵커브리핑 이후 이뤄진 양국 장관 간의 회동에서는 어떤 얘기가 오갔습니까. 기자이창양 산업통상자원부 장관과 지나 러몬도 미국 상무부 장관은 회담을 가진 후 반도체 법과 관련해 공동선언문을 발표했는데요.  반도체 법을 이행하는 과정에서 중국 투자가 제한되고 또 과도한 기업의 정보 제공이 우려되는 가운데 국내 기업의 불확실성을 최소화해 기업에게 우호적인 환경을 조성하기로 합의했습니다.  또한.  이창양 장관은 이 자리에서 IRA에 관련해 크게 3가지를 요청했는데요.  오는 2025년부터 해외우려기업에서 조달하는 핵심광물이 배터리에 포함되면 세액공제 혜택을 받을 수 없는 상황에서 P사 기업을 빠르게 정해줄 것과 투자세액공제 적용 시 국내 기업 우선 고려.  핵심광물 FTA 국가 확대를 당부했습니다.  확대 국가로는 인도네시아와 아르헨티나가 거론됩니다.  양국 모두 미국과 FTA를 체결하지 않았지만.  각각 니켈과 리튬이 풍부한데요.  포스코홀딩스는 아르헨티나에 내년 준공을 목표로

익명화 진행 중:  99%|█████████▉| 85028/85771 [09:23<00:05, 141.31it/s]

Row 85000 기업명 -> 익명화 이름 매핑: {'삼성전자': 'V사', '포스코 홀딩스': 'T사'}
Row 85000 after anonymization - 기업명: T사, full_text: “수급 앞 장사 없다”…外人 역대급 매도 ‘포홀’ 주가가 역대급 매수 ‘삼전’보다 5배 더 오른 이유?  [투자360  회전율.  V사 36.  85.  vs 포스코홀딩스 250.  45. 31일 한국거래소에 따르면 올해 첫 거래일. 1월 2일. 부터 8월 29일까지 종목별 거래대금 순위에서 144조9212억원을 기록한 삼성전자가 1위를 차지했다.  바로 뒤 2위는 97조3901억원의 포스코홀딩스 몫이었다.  두 종목이 종목별 거래대금 최상위에 이름을 올린 것은 올해 국내 증시에서 . 큰손.  역할을 했던 개인. 외국인 투자자들의 활발한 거래 참여 덕분이다.  삼성전자의 경우 외국인 투자자 순매수액 1위. 12조9601억원.  종목이자 개인 투자자 순매도액 1위. 9조5687억원.  종목에 이름을 올렸다.  포스코홀딩스는 반대로 외국인 투자자 순매도액 1위. 8조9532억원. .  개인 투자자 순매수액 1위. 10조503억원. 였다.  주목할 점은 두 종목 모두 활발한 거래에도 불구하고 주가 변동률에서는 큰 격차가 보인다는 점이다.  연초 대비 기준 시점. 8월 29일. 까지 V사 주가가 20.  80.  상승할 때 포스코홀딩스의 주가는 무려 110.  85. 가 올랐기 때문이다.  두 종목의 주가 상승률 간 격차는 5.  3배에 이른다.  이런 차이를 만든 가장 큰 요인으로는 바로 . 수급. 이 꼽힌다.  연초 대비 지난 29일까지 삼성전자와 포스코홀딩스 주식의 회전율은 각각 36.  85. .  250.  45. 였다.  포스코홀딩스의 회전율이 삼성전자의 무려 6.  8배 수준이었다.  투자자별로 좀 더 자세히 회전율을 계산해 보면 수급의 격차는 더 분명히 드러난다.  포스코홀딩스에 대해 10조원 넘게 순매수에 나섰던 개인들의 회전율은 311.  94. 로 외

익명화 진행 중: 100%|██████████| 85771/85771 [09:28<00:00, 150.95it/s]



익명화된 데이터프레임:
                              title                 date  \
0        스페셜 리포트] 한국에는 ‘CEO 시장’이 없다  2019-02-08 19:59:57   
1      고의 공시누락?…근거없다 법정서 맞붙은 삼바-증선위  2019-02-11 18:45:50   
2   사라지는 로또아파트·깨지는 분양공식…'대마불사'는 통할까  2019-02-26 10:38:00   
3    한진칼에만 최소한 경영참여…반쪽짜리 절충안 나온 이유는  2019-02-01 15:28:05   
4  마켓뷰] 뒷심 발휘한 코스피…1월에만 3.4조원 산 외국인  2019-01-29 16:44:06   

                                             content category  \
0  두 부문은 2014년까지 통합 운영돼오다가 2015년 각 부문의 전문성 제고를 위해...       정치   
1  이날 심문은 삼성바이오로직스가 증선위를 상대로 지난해 10월 행정소송을 제기한데 따...       사회   
2  지난 달에는 서울에서는 무조건 청약이 마감된다는 . 서울불패.  신화가 깨졌다.  ...       경제   
3  1일 국민연금의 최고의사결정기구인 기금운용위원회는 2019년 제2차 회의를 열고 한...       사회   
4  반면 코스닥지수는 4거래일 연속 오름세를 멈추고 숨고르기를 택했다.  29일 코스피...       경제   

                                           full_text  cluster  evaluation_1  \
0  스페셜 리포트] 한국에는 ‘CEO 시장’이 없다  두 부문은 2014년까지 통합 운...       10           0.1   
1  고의 공시누락?…근거없다 법정서 맞붙은 삼바-증선위  이날 심문은 삼성바이오로직스가...       17           0.1 

In [ ]:
df.to_csv('/content/drive/MyDrive/Kwargs/적합성/anonymized_data.csv', encoding='utf-8', index=False)

In [ ]:
df.shape

(85771, 8)

In [ ]:
df.sample(20)

title                 date  \
24775                  종합]카카오, 3조 클럽 입성…카톡에 광고입힌 승부수 통했다  2020-02-13 09:58:17   
3517                삼성, 이번엔 ‘입법리스크’… ‘3%룰’ 통과땐 전자주식 강제매각  2021-05-03 11:51:23   
36733  DMZ documentary festival to go ahead on-site d...  2020-09-02 16:52:10   
23165                  침수 지하차도 회피 등 '안전도로' 안내하는 내비는 없을까?  2023-09-26 16:31:09   
65638    단독]“삼성물산 자사주 소각해라”…상장 후 8년째 멈춘 주가에 주주서한 보낸 슈퍼개미  2022-03-07 06:01:05   
81046                      K바이오 뜬다②]무섭게 크고 있다…선두 주자는 어디?  2023-08-14 06:02:05   
49718            최기성의 허브車]킹스맨·아이언맨·아바타, ‘신차 개발 혁명’ 일으키다   2019-12-19 12:17:49   
31488                위기의 한국 금융]보험료 '0%성장' 먹구름, 매물도 안 팔린다  2020-06-19 11:13:07   
31517                        모바일에 서툰 고령층 위해 '목소리 서비스' 제공  2020-06-04 07:04:10   
52117                      뉴스새벽배송]소비심리 부진 우려에 나스닥 2.98%↓  2022-06-29 08:00:01   
24166                 문답]이석우 FATF 가이드라인, 정부 코인 규제 촉매제될 것  2019-09-04 13:45:52   
65213                                 SK하이닉스, 중국에 혹시 당했나  2021-12-30 03:02:01   
49322                      현대제철 비정규직 노조 차별 시정 없으면 29일 파업  2019-10-15 14:15:07   
20731                        민원기의 디지털경제]세계 최고의 창업국가를 만들자  2021-07-14 16:02:15   
46878                   전해액 소량 사용 고성능 발휘 리튬-황 차세대 배터리 개발  2023-01-10 11:11:22   
55735                   코로나로 멈춘 상반기 채용, 현대차·삼성·산은이 물꼬 텄다  2020-04-10 09:33:03   
34592                         재밌고 유용해~…하나은행 '아이부자' 흥행 성공  2023-05-23 10:22:05   
2738                '대장주' 삼성 주가, 당장은 덤덤하지만 지배구조 판 바뀌면...  2020-10-26 04:31:17   
68531  ‘팔수록 적자’ 낸드플래시도 반등 조짐… 삼성전자·SK하이닉스, 4분기 실적 개선 ...  2023-11-01 16:10:05   
50417                             현대차그룹株, 맏형 달릴때 아우들 게걸음  2020-08-31 11:12:25   

                                                 content category  \
24775  영업이익 역시 대폭 증가했다.  주요 사업부문 역시 고르게 성장하며 매출 확대와 수...     IT과학   
3517   재계와 학계는 . 지배구조가 간신히 안정을 찾았는데 법 통과 시 최악의 경우 삼성전...       경제   
36733  17. 24 in Goyang.  Gyeonggi Province.  just no...       세계   
23165  지하차도의 완전 침수까지 걸린 시간은 고작 10분이었다.  당시 지하차도를 지나던 ...       경제   
65638  2020년말 기준 이 부회장이 17.  48. 를 가진 최대주주며 이부진. 서현 사...       경제   
81046  실제로 투심이 얼어붙기 전인 2021년.  국내 벤처캐피탈 신규 투자 총 7조680...       사회   
49718  면적은 350만. . 105만평. 로 신도시 수준이다.  설계동.  디자인동.  파...     생활문화   
31488  신종 코로나바이러스감염증. 코로나19.  확산으로 국내는 물론 글로벌 금융까지 동반...       경제   
31517  금융권 최초로 FIDO. 생체인식기술로 개인인증을 수행하는 기술.  인증기관과의 시...       경제   
52117  이에 다시 경기침체에 대한 공포가 부각하며 지수는 약세로 장을 마쳤다.  독일 검찰...       경제   
24166  이어 그는 국회가 어서 관련법을 만들어 생태계 활성화에 적극 나서야 해외시장을 따라...       경제   
65213  미. 중 갈등 속에서 반도체 업계 핵심 사업자인 SK하이닉스가 중국에서 생산량 확대...     IT과학   
49322  이들은 또 지난달 20일 광주고법이 순천공장 비정규직 노동자 161명에 대해 전원 ...       사회   
20731  그렇다면 우리나라의 창업환경은 어떤 수준일까.  놀랍게도 US뉴스월드리포트의 . 2...     IT과학   
46878  리튬. 황 배터리의 이론적 에너지밀도는 2567Whkg로서 리튬이온 배터리의 약 1...       사회   
55735  현대자동차는 오는 12일까지 연구개발. RD. .  전략지원 부문에서 신입사원을 수...       사회   
34592  아이부자는 부모와 자녀가 각각 스마트폰에 앱을 설치하고 용돈을 주고 받고 관리할 수...     IT과학   
2738   증권업계에선 단기적으로 주가에 미칠 영향이 크지 않을 것이란 전망이 우세하다.  다...       경제   
68531  1일 시장조사업체 트렌드포스에 따르면 내장형멀티미디어카드. eMMC. 와 범용플래시...     IT과학   
50417  31일 한국거래소에 따르면 현대차는 지난 28일 종가. 17만2000원.  기준 시...       경제   

                                               full_text  cluster  \
24775  종합]R사, 3조 클럽 입성…카톡에 광고입힌 승부수 통했다  영업이익 역시 대폭 증...        2   
3517   삼성, 이번엔 ‘입법리스크’… ‘3%룰’ 통과땐 전자주식 강제매각  재계와 학계는 ...        2   
36733  DMZ documentary festival to go ahead on-site d...        1   
23165  침수 지하차도 회피 등 '안전도로' 안내하는 내비는 없을까?  지하차도의 완전 침수...       16   
65638  단독]“I사 자사주 소각해라”…상장 후 8년째 멈춘 주가에 주주서한 보낸 슈퍼개미 ...       16   
81046  K바이오 뜬다②]무섭게 크고 있다…선두 주자는 어디?  실제로 투심이 얼어붙기 전인...       10   
49718  최기성의 허브車]킹스맨·아이언맨·아바타, ‘신차 개발 혁명’ 일으키다   면적은 3...        0   
31488  위기의 한국 금융]보험료 '0%성장' 먹구름, 매물도 안 팔린다  신종 코로나바이러...        0   
31517  모바일에 서툰 고령층 위해 '목소리 서비스' 제공  금융권 최초로 FIDO. 생체인...        8   
52117  뉴스새벽배송]소비심리 부진 우려에 나스닥 2.98%↓  이에 다시 경기침체에 대한 ...        6   
24166  문답]이석우 FATF 가이드라인, 정부 코인 규제 촉매제될 것  이어 그는 국회가 ...       13   
65213  E사, 중국에 혹시 당했나  미. 중 갈등 속에서 반도체 업계 핵심 사업자인 SK하...        7   
49